# Eksploracja danych z Have Your Say

## Co wyniknęło z eksploracji? 
- Podział danych z HYS na państwa nie ma sensu - brak reprezentatywnej próbki. Sensowny jest natomiast podział na typy aktorów. 
- Odrzucono pliki dotyczące Europejskiego Paktu na rzecz Klimatu ze względu na brak możliwości połączenia PDF-ów z metadanymi w ramce. 
- Należy poprawić odczytywanie tekstów z PDF-ów związanych z konsultacjami odnośnie Planu w zakresie celów klimatycznych - problem z kodowaniem. Wiele spośród tych danych to również jedynie wykresy. 
- Najsensowniejszym źródłem danych okazały się opinie dot. Prawa o klimacie, których jest 931 (niemal 160 po angielsku) i stanowią odpowiednią próbkę w podziale na typy aktorów. 

## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
!python -m spacy download en_core_web_md
# trzeba uruchomić ponownie środowisko wykonawcze po pobraniu

In [ ]:
!pip install PyPDF2

In [ ]:
! pip install swifter
! pip install matplotlib==3.1.3

In [ ]:
en = spacy.load("en_core_web_md")

In [ ]:
from PyPDF2 import PdfFileReader
import os
import swifter
import pickle
from collections import Counter
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

In [ ]:
DIR = '/content/drive/MyDrive/NLP-klimat/'

## Dane z Have Your Say

Wczytanie dokumentów załączonych do uwag otrzymanych w ramach konsultacji publicznych dotyczących trzech inicjatyw publicznych: 
- Europejski Pakt na rzecz Klimatu - `climate_pact`
> Sposobem na promowanie powszechnej mobilizacji społecznej jest „pakt na rzecz klimatu”, który ma zjednoczyć obywateli, przedsiębiorców, społeczeństwo obywatelskie i organy publiczne.

- Plan w zakresie celów klimatycznych na 2030 r. - `climate_plan`
> Celem konsultacji jest zebranie opinii na temat pożądanego poziomu ambicji polityki w zakresie klimatu i energii, niezbędnych działań w różnych sektorach oraz konkretnych strategii politycznych służących zwiększeniu poziomu ambicji w dziedzinie klimatu do 2030 r. Konsultacje mają również na celu zebranie dalszych informacji, w tym planów działania, wytycznych politycznych i badań dotyczących dalszej redukcji emisji gazów cieplarnianych.

- Europejskie prawo o klimacie - `climate_law`
> Europejskie prawo o klimacie stanowi gwarant tego, że wszystkie polityki Unii Europejskiej, sektory gospodarki i grupy społeczne będą dążyć do realizacji celu Europejskiego Zielonego Ładu, ale też tego, że przejście na neutralność klimatyczną będzie nieodwracalne. 



Najpierw przystąpimy do analizy pierwszych dwóch dokumentów, gdyż dla nich dostępne są do pobrania wyniki konsultacji w formie pliku `.csv` i pdf-owych załączników.

#### Wczytanie danych

In [ ]:
climate_pact_files_list = os.listdir(DIR+'attachments-EuropeanClimatePact')
climate_plan_files_list = os.listdir(DIR+'attachments-eu_climate_ambition_2030')

In [ ]:
climate_pact_consultation = []
for file_name in tqdm(climate_pact_files_list):
  file = open(DIR+'attachments-EuropeanClimatePact/'+file_name, 'rb')
  fileReader = PdfFileReader(file)
  num_pages = fileReader.numPages
  count = 0
  text = ""
  #The while loop will read each page.
  while count < num_pages:
      pageObj = fileReader.getPage(count)
      count +=1
      text += pageObj.extractText()
  climate_pact_consultation.append(text)


In [ ]:
climate_plan_consultation = []
for file_name in tqdm(climate_plan_files_list):
  file = open(DIR+'attachments-eu_climate_ambition_2030/'+file_name, 'rb')
  fileReader = PdfFileReader(file)
  num_pages = fileReader.numPages
  count = 0
  text = ""
  #The while loop will read each page.
  while count < num_pages:
      pageObj = fileReader.getPage(count)
      count +=1
      text += pageObj.extractText()
  climate_plan_consultation.append(text)


In [ ]:
climate_pact_df = pd.DataFrame({"filename": climate_pact_files_list, "text": climate_pact_consultation})
climate_plan_df = pd.DataFrame({"filename": climate_plan_files_list, "text": climate_plan_consultation})

#### Analiza wstępna

##### Pakt
Niestety w przypadku Paktu identyfikatory załączonych dokumentów (obecne w nazwach plików PDF) nie są zgodne z identyfikatorami z tabeli opisującej odpowiedzi ankietowanych biorących udział w konsultacjach, co uniemożliwia jednoznaczną identyfikację. 

In [ ]:
climate_pact_contributions = pd.read_csv(DIR+'contributions-EuropeanClimatePact.csv', sep=";", on_bad_lines='skip')
climate_pact_contributions.Reference

In [ ]:
climate_pact_df.filename.str.partition("_")[0]

##### Plan

Dla Planu ID załączników są na szczęście odpowiednie, więc można połączyć załączniki z informacjami z tabeli. 

In [ ]:
climate_plan_contributions = pd.read_csv(DIR+'contributions-eu_climate_ambition_2030.csv', sep=";", on_bad_lines='skip')

In [ ]:
climate_plan_df["Reference"] = climate_plan_df.filename.str.partition("-")[0]
climate_plan_df = climate_plan_df.set_index("Reference").join(climate_plan_contributions.set_index("Reference"), how='left').reset_index()

In [ ]:
climate_plan_df = pd.read_csv(DIR+"climate_plan_df.csv")

In [ ]:
len(climate_plan_df)

Mamy 359 załączników PDF połączonych z odpowiedziami w ankiecie.

Niestety podział na kraje jest niemożliwy ze względu na niewielką liczność danych jeśli chodzi o załączone PDF, w szczególności po angielsku. Wykorzystamy więc podział na typy aktorów.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 10))
i, j = 0, 0
for var in ["Language", "Country", "User type", "Organisation size"]:
  sns.countplot(ax=axs[i][j], y=var, data=climate_plan_df, 
                order=climate_plan_df[var].value_counts().iloc[:10].index,
                color="grey")
  j += 1
  if j > 1: 
    i+=1
    j=0
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 10))
i, j = 0, 0
for var in ["Language", "Country", "User type", "Organisation size"]:
  sns.countplot(ax=axs[i][j], y=var, data=climate_plan_df[climate_plan_df.Language == "English"], 
                order=climate_plan_df[var].value_counts().iloc[:10].index,
                color="grey")
  j += 1
  if j > 1: 
    i+=1
    j=0
fig.tight_layout()
plt.show()

Podejrzany rozkład w przypadku Belgii sugeruje, by sprawdzić, czy wszystko jest w porządku. Po obejrzeniu danych dokładniej okazuje się, że do jednej odpowiedzi może być kilka załączników.  

In [ ]:
climate_plan_df.Reference.value_counts()

##### Prawo

In [ ]:
climate_law_df = pd.read_csv(DIR+'climate_law_feedback.csv')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 10))
i, j = 0, 0
for var in ["language", "country", "userType", "companySize"]:
  sns.countplot(ax=axs[i][j], y=var, data=climate_law_df, 
                order=climate_law_df[var].value_counts().iloc[:10].index,
                color="grey")
  j += 1
  if j > 1: 
    i+=1
    j=0
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 10))
i, j = 0, 0
for var in ["language", "country", "userType", "companySize"]:
  sns.countplot(ax=axs[i][j], y=var, data=climate_law_df[climate_law_df.language =="EN"], 
                order=climate_law_df[var].value_counts().iloc[:10].index,
                color="grey")
  j += 1
  if j > 1: 
    i+=1
    j=0
fig.tight_layout()
plt.show()

### Analiza tekstów

#### Analiza dla Prawa

In [ ]:
climate_law_df = climate_law_df[climate_law_df.language == "EN"]

In [ ]:
tqdm.pandas()
climate_law_docs = climate_law_df['feedback'].swifter.apply(en)

In [ ]:
# eksport przetworzonych dokumentów
with open(DIR + 'climate_law_docs.pickle', 'wb') as f:
  pickle.dump(climate_law_docs, f)

In [ ]:
# eksport przetworzonych dokumentów
with open(DIR + 'climate_law_docs.pickle', 'rb') as f:
  climate_law_docs = pickle.load(f)

In [ ]:
plt.figure(figsize=(15,6))
climate_law_df["length"] = climate_law_docs.str.len()
climate_law_df["length"].hist(bins = 50)
plt.show()

In [ ]:
def plot_counter(counter: Counter, orient: str = 'h', color: str='lightblue', figsize: tuple=(20,13)):
  plt.figure(figsize=figsize)
  keys = [k[0] for k in counter]
  vals = [int(k[1]) for k in counter]
  ax = sns.barplot(x=vals, y=keys, orient=orient, color=color)
  return ax

In [ ]:
climate_law_df["lemmas"] = climate_law_docs.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if token.is_alpha])
climate_law_counter = Counter(climate_law_df["lemmas"].sum()).most_common(30)

In [ ]:
plot_counter(climate_law_counter)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words = en.Defaults.stop_words, ngram_range=(2, 2))
count_vector = cv.fit_transform(climate_law_df['feedback'].values)
sum_ngram = count_vector.sum(axis=0)
ngram_freq = [(ngram, sum_ngram[0, idx]) 
              for ngram, idx in cv.vocabulary_.items()]
ngram_freq = sorted(ngram_freq, key = lambda x: x[1], reverse=True)

In [ ]:
plot_counter(ngram_freq[:20])
plt.show()

In [ ]:
cv3 = CountVectorizer(stop_words = en.Defaults.stop_words, ngram_range=(3, 3))
count_vector3 = cv3.fit_transform(climate_law_df['feedback'].values)
sum_ngram3 = count_vector3.sum(axis=0)
ngram_freq3 = [(ngram, sum_ngram3[0, idx]) 
              for ngram, idx in cv3.vocabulary_.items()]
ngram_freq3 = sorted(ngram_freq3, key = lambda x: x[1], reverse=True)

In [ ]:
plot_counter(ngram_freq3[:20])
plt.show()

#### Analiza dla Planu 

Niestety w przypadku odpowiedzi dotyczących Planu nie udało się poprawnie odczytać PDF-ów, wiele z nich jest po prostu pojedynczymi wykresami, więc ewentualna praca z tymi danymi wymaga jeszcze ręcznego przejrzenia dokumentów. 

In [ ]:
tqdm.pandas()
climate_plan_docs = climate_plan_df['text'].swifter.apply(en)

In [ ]:
# eksport przetworzonych dokumentów
with open(DIR + 'climate_plan_docs.pickle', 'wb') as f:
  pickle.dump(climate_plan_docs, f)

In [ ]:
plt.figure(figsize=(15,6))
climate_plan_df["length"] = climate_plan_docs.str.len()
climate_plan_df["length"].hist(bins = 50)
plt.show()

In [ ]:
climate_plan_df["necp_lemmas"] = climate_plan_docs.apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop if not token.is_punct if token.is_alpha])
climate_plan_counter = Counter(climate_plan_df["necp_lemmas"].sum()).most_common(30)

In [ ]:
plot_counter(climate_plan_counter)
plt.show()

Konieczne jest poprawienie wczytywania PDF. 